In [39]:
import pandas as pd
# pd.set_option('display.width', 1000) 
# pd.set_option('display.max_colwidth', 1000) 
import numpy as np
from chromadb.utils.embedding_functions import OllamaEmbeddingFunction
from chromadb.config import Settings
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, RegexpTokenizer
from langdetect import detect
from gensim.models import Word2Vec
import spacy
import nltk
import re
import ast
import chromadb
import string
import uuid

In [7]:
french_stopwords = set(stopwords.words('french'))

In [41]:
def dict_to_ordered_string(d):
    ordered_string = ""
    for key in sorted(d.keys()):
        ordered_string += f"{key}: {', '.join(filter(None, d[key]))} \n "
    return ordered_string.strip()

In [42]:
dict_to_ordered_string(ast.literal_eval(formation["mst_program"][0]))

'Semestre 1:  Statistique mathématique (inférence 1),  Programmation avancée,  Compléments d’analyses,  Compléments d’algèbres,  Analyse Numérique Matricielle et Calcul Parallèle,  Équation différentielle stochastique (EDS) \n Semestre 2:  Théorie des EDP,  Statistiques décisionnelles et simulation,  Analyse fonctionnelle appliquée I,  Les éléments finis et calculs scientifiques,  Optimisation et contrôle optimale,  Anglais Scientifique I (Soft Skills) \n Semestre 3 - SA:  Analyses de données & BIG DATA,  ANOVA & Modèles MIXTES,  Techniques de Sondages,  Statistique des valeurs extrêmes,  Séries chronologiques,  Anglais Scientifiques II (Soft Skills) \n Semestre 3 - SM:  Homogénéisation et Analyses asymptotiques,  Modélisation 1 et Système dynamiques,  Réseaux de Neurones et Apprentissage Automatique & Programmation avancée 2,  Espaces Lebesgue et Sobolev & Méthode variationelle,  Analyses numériques des systèmes hyperboliques & Méthodes des volumes finis,  Analyse numérique'

In [8]:
article = pd.read_csv("fstt-articles.csv")
clubs = pd.read_csv("fstt-clubs-info.csv")
dep = pd.read_csv("fstt-departements-info.csv")
formation = pd.read_csv("fstt-formation-initial.csv")

In [30]:
general_info = pd.read_csv("general_info.csv", encoding='ISO-8859-1')
general_info.columns=["info", "metadata"]

In [31]:
# general_info = general_info.head(41)
general_info['info'] = general_info['info'].str.replace('\n', ' ')
general_info['info'] = general_info['info'].str.replace('Ã‰', 'é')
general_info['info'] = general_info['info'].str.replace('ï¿½', 'é')
general_info['info'] = general_info['info'].str.replace('Ã©', 'é')
general_info['info'] = general_info['info'].str.replace('â€™', "'")
general_info['info'] = general_info['info'].str.replace('Ã´', "o")

general_info['metadata'] = general_info['metadata'].str.replace('Ã‰', 'é')
general_info['metadata'] = general_info['metadata'].str.replace('ï¿½', 'é')
general_info['metadata'] = general_info['metadata'].str.replace('Ã©', 'é')
general_info['metadata'] = general_info['metadata'].str.replace('â€™', "'")
general_info['metadata'] = general_info['metadata'].str.replace('Ã´', "o")

In [32]:
general_info

,info,metadata
0,PRESENTATION DE LA FST DE TANGER : La Faculté ...,PRESENTATION DE LA FST DE TANGER
1,La FST Tanger comporte de 9 les départements s...,les departements de FSTT
2,La FST de Tanger a pour mission de dispenser l...,les diplomes de FST Tanget
3,Le nom du Doyen de FSTT est Pr.MUSTAPHA DIANI,le Doyen de FSTT
4,La Faculté des sciences et techniques de Tange...,La Faculté des sciences et techniques de Tange...
5,FSTT = FST Tanger = Faculte des Sciences et Te...,FSTT = FST Tanger = Faculté des Sciences et Te...
6,La Faculté des Sciences et Techniques de Tange...,PRÃSENTATION GÃNÃRALE en brief sur FST Tanger
7,Formation Initiale : La FST de Tanger offre de...,Les Formations Initiale de FSTT
8,Formation continue : Les programmes de la f...,Les Formation Contunie de FSTT
9,Recherche scientifique : La recherche scienti...,Les Recherche scientifique dans FSTT


In [12]:
clubs.rename(columns={'departement_info': 'club_description'}, inplace=True)

In [13]:
article.head()

,No,post_title,publish_date,post_content,post_link
0,1,CYBERSEC360° : PERSPECTIVES EN CYBERSÉCURITÉ A...,"mars 26, 2024",Un séminaire sera organisé le le samedi 30 Mar...,https://fstt.ac.ma/Portail2023/cybersec360-per...
1,2,PRÉSENTATION CONCOURS D’INNOVATION TDC,"mars 26, 2024",À la suite de l’appel à candidature au concour...,https://fstt.ac.ma/Portail2023/presentation-co...
2,3,SÉMINAIRE : WATER RESOURCES MANAGEMENT FOR SUS...,"mars 26, 2024","À l’occasion de la Journée mondiale de l’eau, ...",https://fstt.ac.ma/Portail2023/seminaire-water...
3,4,ORACLE MOROCCO R&D CENTER : PROGRAMME DE STAGE...,"mars 23, 2024",Oracle Morocco R&D Center lance l’appel aux ca...,https://fstt.ac.ma/Portail2023/oracle-morocco-...
4,5,APPEL À PROJET : «TERRITORY DEVELOPMENT CHALLE...,"mars 22, 2024",Le Centre régional d’investissement (CRI-TTA) ...,https://fstt.ac.ma/Portail2023/appel-a-projet-...


In [14]:
# Create a new column by merging all columns with a formatted string
article['merged_content1'] = article.apply(
    lambda row: f"titre de post ou article : {row['post_title']} | contenu post ou article : {row['post_content']} | lien de post ou article : {row['post_link']}",
    axis=1
)

In [15]:
# Create a new column by merging all columns with a formatted string
article['merged_content2'] = article.apply(
    lambda row: f"post : {row['post_title']} | contenu du post : {row['post_content']}",
    axis=1
)

In [16]:
# Display the DataFrame
print(article['merged_content1'][0])

titre de post ou article : CYBERSEC360° : PERSPECTIVES EN CYBERSÉCURITÉ AVANCÉE | contenu post ou article : Un séminaire sera organisé le le samedi 30 Mars 2024 (11h-13h) au profit des étudiants du Master Sécurité IT et Big Data et aux étudiants 2ème Année du cycle ingénieur LSI pour présenter et discuter les dernières tendances et les meilleures pratiques en matière de cybersécurité.
Le séminaire sera présenté par le professeur Hicham GIBET TANI, professeur à Faculté Polydisciplinaire de Larache, consultant et spécialiste en cybersécurité, sous le titre : CyberSec360°  Perspectives en Cybersécurité Avancée. Il propose une plongée approfondie dans les domaines complexes des concepts de cybersécurité avancée. | lien de post ou article : https://fstt.ac.ma/Portail2023/cybersec360-perspectives-en-cybersecurite-avancee/


In [17]:
# Display the DataFrame
print(article['merged_content2'][0])

post : CYBERSEC360° : PERSPECTIVES EN CYBERSÉCURITÉ AVANCÉE | contenu du post : Un séminaire sera organisé le le samedi 30 Mars 2024 (11h-13h) au profit des étudiants du Master Sécurité IT et Big Data et aux étudiants 2ème Année du cycle ingénieur LSI pour présenter et discuter les dernières tendances et les meilleures pratiques en matière de cybersécurité.
Le séminaire sera présenté par le professeur Hicham GIBET TANI, professeur à Faculté Polydisciplinaire de Larache, consultant et spécialiste en cybersécurité, sous le titre : CyberSec360°  Perspectives en Cybersécurité Avancée. Il propose une plongée approfondie dans les domaines complexes des concepts de cybersécurité avancée.


In [18]:
# Create a new column by merging all columns with a formatted string
clubs['merged_content1'] = clubs.apply(
    lambda row: f"nom du club : {row['club_name']} | description du club  : {row['club_description']} | lien du club : {row['club_link']}",
    axis=1
)

In [19]:
# Create a new column by merging all columns with a formatted string
clubs['merged_content2'] = clubs.apply(
    lambda row: f"'{row['club_name']}' est un club dans FST Tanger, ce ca une description sur le club  : {row['club_description']}",
    axis=1
)

In [20]:
clubs['merged_content1'][0]

'nom du club : GREENOLOGY | description du club  : Le Club de l’environnement de la FST de Tanger, Dénommé Greenology Club, est nouvellement créé et il a comme vocation de : – Transmettre aux étudiants, une conscience nationale éclairée, un patriotisme activé pour un environnement préservé. – Sensibiliser et rendre les étudiants de la FST de Tanger des combattants contre l’éparpillement de l’environnement. – Tous unis, bien convaincus que nos richesses environnementales très précieuses sont menacées, et qu’avec une conscience large du désastre écologique qui frappe à la porte de notre pays, on pourra gagner la bataille et sauver ce qu’il reste encore à sauver.\nla page facebook:\nhttps://www.facebook.com/greenology.club\nEmail de contact :\nResponsable d’activités 4saaddine@gmail.com | lien du club : https://fstt.ac.ma/Portail2023/greenology/'

In [21]:
clubs['merged_content2'][0]

"'GREENOLOGY' est un club dans FST Tanger, ce ca une description sur le club  : Le Club de l’environnement de la FST de Tanger, Dénommé Greenology Club, est nouvellement créé et il a comme vocation de : – Transmettre aux étudiants, une conscience nationale éclairée, un patriotisme activé pour un environnement préservé. – Sensibiliser et rendre les étudiants de la FST de Tanger des combattants contre l’éparpillement de l’environnement. – Tous unis, bien convaincus que nos richesses environnementales très précieuses sont menacées, et qu’avec une conscience large du désastre écologique qui frappe à la porte de notre pays, on pourra gagner la bataille et sauver ce qu’il reste encore à sauver.\nla page facebook:\nhttps://www.facebook.com/greenology.club\nEmail de contact :\nResponsable d’activités 4saaddine@gmail.com"

In [22]:
# Remove newline characters from 'departement_info' column
dep['departement_info'] = dep['departement_info'].str.replace('\n', ' - ')

In [23]:
dep.head()

,No,departement_name,departement_info
0,1,GÉNIE INFORMATIQUE,Chef : Pr.EL BRAK Mohamed - Email : melb...
1,2,GÉNIE CHIMIQUE,Chef : Pr.CHABBI Mohamed - Email : mchabbi@uae...
2,3,SCIENCES DE LA TERRE,Chef : Pr.BOULAASSAL Hakim - Email : h.boulaas...
3,4,GÉNIE MÉCANIQUE,Chef : Pr.ELAYACHI Ilham - - Email :...
4,5,SCIENCES DE LA VIE,Chef : Pr.HASSANI ZERROUK Mounir - Email : mha...


In [24]:
# Create a new column by merging all columns with a formatted string
dep['merged_content1'] = dep.apply(
    lambda row: f"nom de departement : {row['departement_name']} | le nom de Coordinnateur avec son email : {row['departement_info']}",
    axis=1
)

In [25]:
# Create a new column by merging all columns with a formatted string
dep['merged_content2'] = dep.apply(
    lambda row: f"'{row['departement_name']}' est un departement dans FST Tanger, et le nom de Coordinnateur avec son email est {row['departement_info']}",
    axis=1
)

In [26]:
dep['merged_content1'][0]

'nom de departement : GÉNIE INFORMATIQUE | le nom de Coordinnateur avec son email : Chef : Pr.EL BRAK Mohamed     - Email :   melbrak@uae.ac.ma'

In [27]:
dep['merged_content2'][0]

"'GÉNIE INFORMATIQUE' est un departement dans FST Tanger, et le nom de Coordinnateur avec son email est Chef : Pr.EL BRAK Mohamed     - Email :   melbrak@uae.ac.ma"

In [28]:
formation.head()

,No,mst_name,formation_type,mst_link,mst_objectif,mst_program,mst_skills,mst_Coord
0,1,MST : Analyse Appliquée et Ingénierie Statistique,mst,https://fstt.ac.ma/Portail2023/mst-analyse-app...,Le Master Analyse Appliquée et Ingénierie Stat...,"{'Semestre 1': ['', ' Statistique mathématique...",Donner une formation mathématique et statistiq...,Coordinnateur pédagogique : Pr.EL HALIMI Rach...
1,2,MST : Bases Cellulaires et Moléculaires en Bio...,mst,https://fstt.ac.ma/Portail2023/mst-bases-cellu...,Le master BCMB vise la formation des spécialis...,"{'Semestre 1': ['', ' Techniques Expérimentale...","Secteur public : chercheur, responsable, ingén...",Coordinnateur pédagogique : Pr.Mohamed NHIRI ...
2,3,"MST : Environnement, Aquaculture et Développem...",mst,https://fstt.ac.ma/Portail2023/mst-environneme...,"Encore peu développée au Maroc, l’aquaculture ...","{'Semestre 1': ['', ' Écosystèmes aquatiques e...",NaN,Coordinnateur pédagogique : Pr.Hassan Er-RAIOU...
3,4,MST : Géoressources Energétiques et Réservoirs,mst,https://fstt.ac.ma/Portail2023/mst-georessourc...,L’activité accrue dans la recherche en géoscie...,"{'Semestre 1': ['', ' Géodynamique et Analyse ...",NaN,Coordinnateur pédagogique : Pr.Mohamed Najib Z...
4,5,MST : Génie Civil,mst,https://fstt.ac.ma/Portail2023/mst-genie-civil/,Le programme vise la formation des cadres en G...,"{'Semestre 1': ['', ' Mathématiques Pour l’Ing...",NaN,Coordinnateur pédagogique : Pr.Dkiouak Rachid ...


In [43]:
# Create a new column by merging all columns with a formatted string
formation['merged_content1'] = formation.apply(
    lambda row: f"nom de formation : {row['mst_name']} | type de formation  : {row['formation_type']} | objective de formation  : {row['mst_objectif']} | programme de formation : {row['mst_program']} | Coordinnateur de formation : {row['mst_Coord']} | link de formation : {row['mst_link']} |",
    axis=1
)

In [44]:
# Create a new column by merging all columns with a formatted string
formation['merged_content2'] = formation.apply(
    lambda row: f"objective de formation : '{row['mst_name']}', est {row['mst_objectif']}.",
    axis=1
)

In [45]:
# Create a new column by merging all columns with a formatted string
formation['merged_content3'] = formation.apply(
    lambda row: f"'{row['mst_name']}' est un formation dans FST Tanger de type : {row['formation_type']}",
    axis=1
)

In [46]:
# Create a new column by merging all columns with a formatted string
formation['merged_content4'] = formation.apply(
    lambda row: f"le programme de formation : '{row['mst_name']}' est : {dict_to_ordered_string(ast.literal_eval(row['mst_program']))}",
    axis=1
)

In [47]:
# Create a new column by merging all columns with a formatted string
formation['merged_content5'] = formation.apply(
    lambda row: f"Le Coordinnateur de formation '{row['mst_name']}' est {row['mst_Coord']}",
    axis=1
)

In [48]:
# Create a new column by merging all columns with a formatted string
formation['merged_content6'] = formation.apply(lambda row: f"link de formation '{row['mst_name']}' est : {row['mst_link']}", axis=1)

In [50]:
formation['merged_content1'][0]

"nom de formation : MST : Analyse Appliquée et Ingénierie Statistique | type de formation  : mst | objective de formation  : Le Master Analyse Appliquée et Ingénierie Statistique (MAAIS) de l'Université Abdelmalek Essaadi a pour vocation d'offrir des parcours qui trouvent leurs bases dans une approche pluridisciplinaire. Au département de mathématiques on propose aux étudiants d’acquérir une double compétence en mathématiques et en statistiques appliquées s’appuyant sur un tronc commun développant des concepts et des outils de base. Pour une approche pluridisciplinaire des problèmes liés aux applications, les mathématiques et les statistiques appliquées ne peuvent que former des étudiants pour la recherche et les activités professionnelles de haut niveau avec un bon bagage mathématique. La spécialisation se ferait au terme du second semestre (S2) selon le choix des étudiants. En plus une bonne formation en tronc commun solide permettra de mieux optimiser les ressources humaines surtout

In [51]:
formation['merged_content2'][0]

"objective de formation : 'MST : Analyse Appliquée et Ingénierie Statistique', est Le Master Analyse Appliquée et Ingénierie Statistique (MAAIS) de l'Université Abdelmalek Essaadi a pour vocation d'offrir des parcours qui trouvent leurs bases dans une approche pluridisciplinaire. Au département de mathématiques on propose aux étudiants d’acquérir une double compétence en mathématiques et en statistiques appliquées s’appuyant sur un tronc commun développant des concepts et des outils de base. Pour une approche pluridisciplinaire des problèmes liés aux applications, les mathématiques et les statistiques appliquées ne peuvent que former des étudiants pour la recherche et les activités professionnelles de haut niveau avec un bon bagage mathématique. La spécialisation se ferait au terme du second semestre (S2) selon le choix des étudiants. En plus une bonne formation en tronc commun solide permettra de mieux optimiser les ressources humaines surtout au niveau des encadrants des PFEs (Master

In [52]:
formation['merged_content3'][0]

"'MST : Analyse Appliquée et Ingénierie Statistique' est un formation dans FST Tanger de type : mst"

In [53]:
formation['merged_content4'][0]

"le programme de formation : 'MST : Analyse Appliquée et Ingénierie Statistique' est : Semestre 1:  Statistique mathématique (inférence 1),  Programmation avancée,  Compléments d’analyses,  Compléments d’algèbres,  Analyse Numérique Matricielle et Calcul Parallèle,  Équation différentielle stochastique (EDS) \n Semestre 2:  Théorie des EDP,  Statistiques décisionnelles et simulation,  Analyse fonctionnelle appliquée I,  Les éléments finis et calculs scientifiques,  Optimisation et contrôle optimale,  Anglais Scientifique I (Soft Skills) \n Semestre 3 - SA:  Analyses de données & BIG DATA,  ANOVA & Modèles MIXTES,  Techniques de Sondages,  Statistique des valeurs extrêmes,  Séries chronologiques,  Anglais Scientifiques II (Soft Skills) \n Semestre 3 - SM:  Homogénéisation et Analyses asymptotiques,  Modélisation 1 et Système dynamiques,  Réseaux de Neurones et Apprentissage Automatique & Programmation avancée 2,  Espaces Lebesgue et Sobolev & Méthode variationelle,  Analyses numériques 

In [54]:
formation['merged_content5'][0]

"Le Coordinnateur de formation 'MST : Analyse Appliquée et Ingénierie Statistique' est Coordinnateur pédagogique  : Pr.EL HALIMI Rachid \xa0 \xa0 \xa0 \xa0 \xa0:\xa0 r.elhalimi@uae.ac.ma"

In [55]:
formation['merged_content6'][0]

"link de formation 'MST : Analyse Appliquée et Ingénierie Statistique' est : https://fstt.ac.ma/Portail2023/mst-analyse-appliquee-etingenierie-statistique/"

In [69]:
formation["merged_content5"][8]

"le coordinnateur de formation 'mst : intelligence artificielle et sciences de données' est coordinnateur pédagogique  : pr.ezziyyani mostafa          :  mezziyyani .ac.ma"

In [56]:
def is_french(text):
    try:
        return detect(text) == 'fr'
    except:
        return False
        
def preprocessing(text):
    # Convert to string and lowercase
    text = str(text).lower()
    
    # Remove URLs
    # text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    
    # Remove mentions
    text = re.sub(r'@\w+', ' ', text)

    text = re.sub(r'\xa0', ' ', text)
    
    # Remove \n
    text = re.sub(r'\n', ' ', text)
    
    # Remove specific unwanted characters
    text = re.sub(r'«|»|“|”|’|‘', ' ', text)
    
    # Check if text is in French
    if not is_french(text):
        return ''

    return text

In [57]:
article["merged_content1"] = article["merged_content1"].apply(preprocessing)
article["merged_content2"] = article["merged_content2"].apply(preprocessing)
clubs["merged_content1"] = clubs["merged_content1"].apply(preprocessing)
clubs["merged_content2"] = clubs["merged_content2"].apply(preprocessing)
dep["merged_content1"] = dep["merged_content1"].apply(preprocessing)
dep["merged_content2"] = dep["merged_content2"].apply(preprocessing)
formation["merged_content1"] = formation["merged_content1"].apply(preprocessing)
formation["merged_content2"] = formation["merged_content2"].apply(preprocessing)
formation["merged_content3"] = formation["merged_content3"].apply(preprocessing)
formation["merged_content4"] = formation["merged_content4"].apply(preprocessing)
formation["merged_content5"] = formation["merged_content5"].apply(preprocessing)
formation["merged_content6"] = formation["merged_content6"].apply(preprocessing)
general_info['info'] = general_info['info'].apply(preprocessing)

In [58]:
article["merged_content2"][0]

'post : cybersec360° : perspectives en cybersécurité avancée | contenu du post : un séminaire sera organisé le le samedi 30 mars 2024 (11h-13h) au profit des étudiants du master sécurité it et big data et aux étudiants 2ème année du cycle ingénieur lsi pour présenter et discuter les dernières tendances et les meilleures pratiques en matière de cybersécurité. le séminaire sera présenté par le professeur hicham gibet tani, professeur à faculté polydisciplinaire de larache, consultant et spécialiste en cybersécurité, sous le titre : cybersec360°  perspectives en cybersécurité avancée. il propose une plongée approfondie dans les domaines complexes des concepts de cybersécurité avancée.'

In [59]:
article.drop(columns="No", inplace=True)
clubs.drop(columns="No", inplace=True)
formation.drop(columns="No", inplace=True)
dep.drop(columns="No", inplace=True)

In [60]:
# create EF with custom endpoint
ef = OllamaEmbeddingFunction(
    model_name="nomic-embed-text",
    url="http://localhost:11434/api/embeddings",
)

In [61]:
# Configure the ChromaDB client with persistence
persist_directory = "/home/idriss/Desktop/chroma_db"
client = chromadb.PersistentClient(path=persist_directory)

In [63]:
# client.delete_collection(name="text_embeddings")

In [62]:
# Create a collection in ChromaDB with the OllamaEmbeddingFunction
collection = client.create_collection(name="text_embeddings")

In [64]:
import uuid

def process_and_store_embeddings(dataframe, column_names, collection):
    ids = []
    metadatas = []
    documents = []

    for idx, row in dataframe.iterrows():
        doc_metadata = {}
        for key, value in row.items():
            if key not in column_names:
                doc_metadata[key] = value  # Leave non-string fields as is
        
        for column in column_names:
            sentence = row[column]
            if sentence is not None and sentence != '':
                ids.append(str(uuid.uuid1()))
                metadatas.append(doc_metadata)
                documents.append(sentence)
    collection.add(
        ids=ids,
        #embeddings=embeddings,
        metadatas=metadatas,
        documents=documents
    )

# Assuming article, clubs, formation DataFrames are already preprocessed
# Store embeddings, metadata, and documents from each DataFrame
process_and_store_embeddings(article, ["merged_content1"], collection)
process_and_store_embeddings(article, ["merged_content2"], collection)
process_and_store_embeddings(clubs, ["merged_content1"], collection)
process_and_store_embeddings(clubs, ["merged_content2"], collection)
process_and_store_embeddings(dep, ["merged_content1"], collection)
process_and_store_embeddings(dep, ["merged_content2"], collection)
process_and_store_embeddings(formation, ["merged_content1"], collection)
process_and_store_embeddings(formation, ["merged_content2"], collection)
process_and_store_embeddings(formation, ["merged_content3"], collection)
process_and_store_embeddings(formation, ["merged_content4"], collection)
process_and_store_embeddings(formation, ["merged_content5"], collection)
process_and_store_embeddings(formation, ["merged_content6"], collection)
process_and_store_embeddings(general_info, ["info"], collection)

print("Embeddings, metadata, and documents stored in ChromaDB successfully.")


Embeddings, metadata, and documents stored in ChromaDB successfully.


In [32]:
collection.peek()

{'ids': ['09389214-21f6-11ef-aa8c-d46d6d576da4',
  '09389215-21f6-11ef-aa8c-d46d6d576da4',
  '09389216-21f6-11ef-aa8c-d46d6d576da4',
  '09389217-21f6-11ef-aa8c-d46d6d576da4',
  '09389218-21f6-11ef-aa8c-d46d6d576da4',
  '09389219-21f6-11ef-aa8c-d46d6d576da4',
  '0938921a-21f6-11ef-aa8c-d46d6d576da4',
  '0938921b-21f6-11ef-aa8c-d46d6d576da4',
  '0938921c-21f6-11ef-aa8c-d46d6d576da4',
  '0938921d-21f6-11ef-aa8c-d46d6d576da4'],
 'embeddings': [[0.001996981678530574,
   -0.01846979558467865,
   -0.04105418175458908,
   -0.10017459839582443,
   -0.008761539123952389,
   -0.007153053767979145,
   0.013611610047519207,
   0.07812973856925964,
   0.02077500894665718,
   0.0571969598531723,
   -0.01889941841363907,
   -0.03595414757728577,
   -0.014790087006986141,
   0.0193627942353487,
   0.0013051670975983143,
   -0.06735706329345703,
   0.01769564300775528,
   -0.03948791325092316,
   -0.016260448843240738,
   -0.002855441765859723,
   0.05213575437664986,
   -0.059626027941703796,
   0.0309

In [65]:
# Load the existing collection
collection = client.get_collection(name="text_embeddings")

# Perform a query
query = "informatique"
results = collection.query(
    query_texts=[query],
    n_results=10
)

for result in results['documents']:
    print(result)


["objective de formation : 'génie informatique', est avoir une culture de base scientifique ;acquérir une base solide dans les axes majeurs et fondamentaux de la discipline informatique : algorithmique, programmation, bases de données, technologies web, systèmes d exploitation et réseaux informatiques, ... ;développer un savoir-faire technique en informatique : technologie objet, informatique distribuée, architectures client-serveur et n-tiers, applications hétérogènes, … ;.", "'génie informatique' est un formation dans fst tanger de type : lst", "link de formation 'génie informatique' est : https://fstt.ac.ma/portail2023/lst-genie-informatique/", "'ingénierie de développement d applications informatiques' est un formation dans fst tanger de type : lst", "le coordinnateur de formation 'génie informatique' est coordinnateur pédagogique  : pr.boudhir anouar abdelhakim             : aboudhir .ac.ma", "objective de formation : 'ingénierie de développement d applications informatiques', est